In [227]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
import seaborn as sns
from matplotlib import style
import math
import random


In [228]:
#LP MAXIMIZATION MODEL

from pulp import *
Promo_Model = LpProblem("Pharma_Promotion_Model", LpMaximize)


In [229]:
#PARAMETERS

T=30 # end of promotion horizon
PHorizon = list(range(0, T))

I=10 # product count 
Products = list(range(0, I))

#NOT USED RIGHT NOW
Promo_Levels = [0,1,2]

Base_Demand = np.zeros((I, T), dtype = int)

for i in Products:
    for t in PHorizon:
        Base_Demand[i][t] = random.randint(0, 10000)
    
Promo_Sensitivity = np.zeros(I, dtype = float)
for i in Products:
    Promo_Sensitivity[i] = random.uniform(1,2)


Total_PaidQty_Limit = 1000000
Total_FreeQty_Limit = 250000

Promo_Sensitivity


array([1.19488356, 1.13720047, 1.65060048, 1.17997442, 1.65451094,
       1.76058517, 1.73279655, 1.88830118, 1.25199053, 1.36807305])

In [230]:
#DECISION VARIABLES

#Paid Quantity
PaidQtyLimit = 20000
P = LpVariable.dicts("Paid_Quantity",(Products,PHorizon),lowBound=0, upBound=PaidQtyLimit, cat='Integer')

#Free Quantity
FreeQtyLimit = 5000
F = LpVariable.dicts("Free_Quantity",(Products,PHorizon),lowBound=0, upBound=FreeQtyLimit, cat='Integer')

#Is Product "i" at Time "t" Promoted?
X = LpVariable.dicts("IsPromoted",(Products,PHorizon),cat='Binary')

#Promo Ratio
#R = LpVariable.dicts("Promo_Ratio",(Products,PHorizon),lowBound=0, upBound=1, cat='Continuous')


In [231]:
#OBJECTIVE FUNCTION

Total_Paid_Quantity = lpSum(lpSum(P[i][t] for i in Products) for t in PHorizon)

Promo_Model += Total_Paid_Quantity


In [232]:
#DEMAND SATISFACTION
for i in Products:
    for t in PHorizon:
        Promo_Model += P[i][t] + F[i][t] <= Base_Demand[i][t]*(1 + Promo_Sensitivity[i]*F[i][t]*(1/Base_Demand[i][t]))

#NO FREE QTY WITHOUT PROMOTION
M = FreeQtyLimit
for i in Products:
    for t in PHorizon:
        Promo_Model += F[i][t] <= X[i][t]*M
        Promo_Model += F[i][t] >= X[i][t]-0.5

#PROMOTED PRODUCTS AT TIME "t"
Promoted_Products = 8
for t in PHorizon:
    Promo_Model += lpSum(X[i][t] for i in Products) <= Promoted_Products
    
#PROMO PERIOD FOR PRODUCT "i"  
Max_Promo_Duration = 25
Min_Promo_Duration = 15
for i in Products:
    Promo_Model += lpSum(X[i][t] for t in PHorizon) <= Max_Promo_Duration
    Promo_Model += lpSum(X[i][t] for t in PHorizon) >= Min_Promo_Duration

#TOTAL PAID QUANTITY PER PRODUCT
for i in Products:
    for t in PHorizon:
        Promo_Model += lpSum(P[i][t] for t in PHorizon) <= Total_PaidQty_Limit
    
#TOTAL FREE QUANTITY PER PRODUCT
for i in Products:
    for t in PHorizon:
        Promo_Model += lpSum(F[i][t] for t in PHorizon) <= Total_FreeQty_Limit


In [233]:
solver = GUROBI()
solver.solve(Promo_Model)

# The status of the solution is printed to the screen
print("Status:", LpStatus[Promo_Model.status])


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.3.0 22D68)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1550 rows, 900 columns and 20700 nonzeros
Model fingerprint: 0xd9125825
Variable types: 0 continuous, 900 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [5e-01, 1e+06]
Found heuristic solution: objective 1476103.0000
Presolve removed 1500 rows and 600 columns
Presolve time: 0.00s
Presolved: 50 rows, 300 columns, 900 nonzeros
Found heuristic solution: objective 2064472.0000
Variable types: 0 continuous, 300 integer (300 binary)
Found heuristic solution: objective 2071042.0000

Root relaxation: objective 2.071468e+06, 99 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | I

In [234]:
# OUTPUT

# OPTIMIZED OBJECTIVE FUNCTION
print("Optimal Total Paid Quantity = ", pulp.value(Promo_Model.objective),"\n")

# PRINTS VARIABLES AND OPTIMAL VALUES
for v in Promo_Model.variables():
    print(v.name, "=", v.varValue)

Optimal Total Paid Quantity =  2071468.0 

Free_Quantity_0_0 = 5000.0
Free_Quantity_0_1 = 5000.0
Free_Quantity_0_10 = 5000.0
Free_Quantity_0_11 = 5000.0
Free_Quantity_0_12 = 5000.0
Free_Quantity_0_13 = 0.0
Free_Quantity_0_14 = 5000.0
Free_Quantity_0_15 = 5000.0
Free_Quantity_0_16 = 5000.0
Free_Quantity_0_17 = 5000.0
Free_Quantity_0_18 = 5000.0
Free_Quantity_0_19 = 0.0
Free_Quantity_0_2 = 0.0
Free_Quantity_0_20 = 0.0
Free_Quantity_0_21 = 5000.0
Free_Quantity_0_22 = 5000.0
Free_Quantity_0_23 = 5000.0
Free_Quantity_0_24 = 5000.0
Free_Quantity_0_25 = 5000.0
Free_Quantity_0_26 = 5000.0
Free_Quantity_0_27 = 5000.0
Free_Quantity_0_28 = 5000.0
Free_Quantity_0_29 = 5000.0
Free_Quantity_0_3 = 5000.0
Free_Quantity_0_4 = 5000.0
Free_Quantity_0_5 = 5000.0
Free_Quantity_0_6 = 0.0
Free_Quantity_0_7 = 5000.0
Free_Quantity_0_8 = 5000.0
Free_Quantity_0_9 = 5000.0
Free_Quantity_1_0 = 5000.0
Free_Quantity_1_1 = 5000.0
Free_Quantity_1_10 = 5000.0
Free_Quantity_1_11 = 5000.0
Free_Quantity_1_12 = 5000.0
Free

In [235]:
#PROMO RATIO IS CALCULATED BASED ON THE OPTIMUM P[i][t] AND F[i][t] VALUES
Promo_Ratio = np.zeros((I, T), dtype = float)

for i in Products:
    for t in PHorizon:
        Promo_Ratio[i][t] = (F[i][t].varValue)/(P[i][t].varValue + F[i][t].varValue)

df_Promo_Ratio = pd.DataFrame(Promo_Ratio,
                 index=range(1,I+1),
                 columns=range(1,T+1))
df_Promo_Ratio

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
1,0.314051,0.392372,0.000000,0.396542,0.422726,0.484590,0.000000,0.543183,0.457750,0.461681,...,0.000000,0.333289,0.322352,0.557165,0.371416,0.346141,0.413121,0.338135,0.402966,0.470367
2,0.401445,0.666311,0.374644,0.000000,0.000000,0.000000,0.319468,0.000000,0.636051,0.000000,...,0.344281,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.542888,0.506483,0.461681,0.313834,0.345447,0.358989,0.417119,0.381214,0.357577,0.000000,...,0.000000,0.321213,0.562430,0.293876,0.300661,0.446229,0.438097,0.371113,0.274665,0.391328
4,0.450897,0.343454,0.569671,0.000000,0.000000,0.350853,0.481742,0.356075,0.362634,0.661638,...,0.565291,0.365925,0.623830,0.491207,0.728969,0.550055,0.559472,0.000000,0.625939,0.000000
5,0.317199,0.564270,0.366193,0.512663,0.416736,0.508854,0.276091,0.549089,0.000000,0.290850,...,0.408030,0.555124,0.298846,0.359971,0.290175,0.278940,0.288767,0.384172,0.465853,0.522302
6,0.469704,0.278102,0.413873,0.307295,0.540132,0.494169,0.000000,0.000000,0.000000,0.315537,...,0.561861,0.423872,0.424160,0.000000,0.421195,0.366032,0.510673,0.462834,0.459263,0.306937
7,0.000000,0.000000,0.354786,0.359350,0.447347,0.289721,0.416840,0.306147,0.353157,0.415489,...,0.531971,0.382614,0.000000,0.309693,0.290782,0.533504,0.370178,0.379248,0.322664,0.352634
8,0.360698,0.490966,0.000000,0.267537,0.453186,0.000000,0.390198,0.267551,0.267838,0.480584,...,0.311449,0.374953,0.272242,0.298276,0.528653,0.000000,0.291070,0.287323,0.000000,0.257852
9,0.000000,0.000000,0.564079,0.372523,0.478652,0.712555,0.694444,0.349797,0.343147,0.559973,...,0.640615,0.313991,0.467028,0.430960,0.648593,0.719942,0.000000,0.316276,0.452489,0.424052
10,0.309272,0.349993,0.301114,0.300264,0.312695,0.495050,0.588651,0.567086,0.669523,0.530729,...,0.308509,0.000000,0.462364,0.339928,0.000000,0.646747,0.594813,0.359170,0.643004,0.652146


In [236]:
Promo_Sensitivity


array([1.19488356, 1.13720047, 1.65060048, 1.17997442, 1.65451094,
       1.76058517, 1.73279655, 1.88830118, 1.25199053, 1.36807305])